# Dynamic damages

In [1]:
import numpy as np
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()) + '/source')
import pickle
from utilities import dLambda
from supportfunctions import finiteDiff
import matplotlib.pyplot as plt

In [2]:
solu = pickle.load(open('../data/solution/solu_dynamicdmg_21*201_0301_16:41', 'rb'))

In [3]:
solu

{'FC_Err': 1,
 'PIThis': array([[[0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542],
         [0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542],
         [0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542],
         ...,
         [0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542],
         [0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542],
         [0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542]],
 
        [[0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542],
         [0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542],
         [0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,
          0.00338542, 0.00338542],
         ...,
         [0.00338542, 0.00338542, 0.00338542, ..., 0.00338542,


In [4]:
solu['y_grid'][100]

2.0

In [5]:
solu['sigma2']

0.002177598558933893

In [24]:
# backward induction
rho = .9
mu2 = 1.86/1000
sigma2 = 0.002177598558933893
error = 1
tol = 1e-8
episode = 0
epsilon = .3

In [6]:
solu_dict = solu['solu_dynamicdmg']
z_grid = solu['z_grid']
y_grid = solu['y_grid']
hz = solu['hz']
hy = solu['hy']
delta = .01
eta = .032
z_mat = solu['z_mat'][:,:21]
y_mat = solu['y_mat'][:,:21]
v_n = solu['v_n']

In [7]:
y_grid_new = y_grid[:101]
gamma2pList = np.array([2*0.0197, 2*0.3853])
v0 = solu_dict[0]['phi'][:, :101]
v1 = solu_dict[gamma2pList[0]]['phi'][:, :101]
v2 = solu_dict[gamma2pList[1]]['phi'][:, :101]
# numy = len(y_new)
numz = len(z_grid)
numy = len(y_grid_new)
numy

101

In [8]:
numdmg = 3
tran_prob = np.ones((numdmg-1, numz, numy))/(numdmg-1)

In [9]:
v1.shape,tran_prob.shape

((21, 101), (2, 21, 101))

In [10]:
(z_mat, y_mat) = np.meshgrid(z_grid, y_grid_new, indexing='ij')
stateSpace = np.hstack([z_mat.reshape(-1,1,order='F'), y_mat.reshape(-1,1,order='F')])

In [11]:
gamma2pMat = np.zeros((numdmg-1, numz, numy))
gamma2pMat[0] = gamma2pList[0]
gamma2pMat[1] = gamma2pList[1]
dlambdaMat = dLambda(y_mat, 1, 0.00017675, 2*0.0022, np.sum(gamma2pMat*tran_prob, axis=0), 2)

In [12]:
dlambdaMat.shape

(21, 101)

In [14]:
target = np.array([v1,v2])
target.shape, dlambdaMat.shape

((2, 21, 101), (21, 101))

In [15]:
V = np.sum(tran_prob*target, axis=0)
terminal_v = v0[:,-1]

In [16]:
V = np.sum(tran_prob*target, axis=0)
terminal_v = v0[:,-1]

In [17]:
V.reshape(len(stateSpace))

array([ 0.04775059,  0.04746675,  0.04718203, ...,  0.0008105 ,
       -0.00047782, -0.00184451])

In [102]:
def pde_solver(phi_grid, z_grid, y_grid, terminal_condition, dlambdaMat, epsilon=.3,
           delta=.01, eta=.032, 
           mu2=1.86/1000, sigma2=0.002177598558933893, rho=.9, ):
    
    n_y = len(y_grid)
    n_z = len(z_grid)
    hy = y_grid[1] - y_grid[0]
    hz = z_grid[1] - z_grid[0]
    LHS = np.zeros((n_y*n_z, n_y*n_z))
    RHS = np.zeros(n_y*n_z)
    for j in range(n_z):
        for i in range(n_y):
            idx = j*n_y + i
            idx_yp1 = idx + 1
            idx_ym1 = idx - 1
            idx_zp1 = (j+1)*n_y + i
            idx_zp2 = (j+2)*n_y + i
            idx_zm1 = (j-1)*n_y + i
            idx_zm2 = (j-2)*n_y + i
            phi = phi_grid[idx]
            z = z_grid[j]
            condition = terminal_condition[j]
            dlambda = dlambdaMat[j,i]
            LHS[idx, idx] += - delta - 1/epsilon
            temp_1 = -rho*(z-mu2)
            var = z*sigma2**2/2
            if j == 0:
                LHS[idx, idx] += -temp_1/hz + var/(hz**2)  
                LHS[idx, idx_zp1] += temp_1/hz - var*2/(hz**2)
                LHS[idx, idx_zp2] += var/(hz**2)
            elif j == n_z-1:
                LHS[idx, idx] += temp_1/hz + var/(hz**2)
                LHS[idx, idx_zm1] += -temp_1/hz - var*2/(hz**2)
                LHS[idx, idx_zm2] += var/(hz**2)            
            else:
                LHS[idx, idx] += - var*2/(hz**2)
                LHS[idx, idx_zp1] += temp_1/(2*hz) + var/(hz**2)
                LHS[idx, idx_zm1] += -temp_1/(hz*2)+ var/(hz**2)
            
            if i == 0:
                dphidy= (phi_grid[idx_yp1]-phi_grid[idx])/hy
                e = - delta*eta/(dphidy*z + (eta-1)*dlambda*z)
                LHS[idx, idx] +=  - z*e/hy
                LHS[idx, idx_yp1] += z*e/hy
                
            elif i >= 1 and i< n_y-1:
                dphidy = (phi_grid[idx_yp1]-phi_grid[idx_ym1])/(2*hy)
                e = - delta*eta/(dphidy*z + (eta-1)*dlambda*z)
                LHS[idx, idx_yp1] += z*e/(2*hy)
                LHS[idx, idx_ym1] += -z*e/(2*hy)
            elif i == n_y-1:
                dphidy = (phi_grid[idx]-phi_grid[idx_ym1])/hy
                e = - delta*eta/(dphidy*z+(eta-1)*dlambda*z)
                LHS[idx, idx] += 1
                LHS[idx, idx] +=   z*e/hy
                LHS[idx, idx_ym1] += - z*e/hy
                RHS[idx] += condition
                
            else:
                raise Exception("error.")
            if e<=0:
                e = 1e-8
            RHS[idx] += -1/epsilon*phi - delta*eta*np.log(e) - (eta-1)*dlambda*z*e 
    phi_grid = np.linalg.solve(LHS, RHS)
    return phi_grid

In [103]:
def false_transient(phi_grid, z_grid, y_grid, terminal_condition, dlambdaMat,epsilon=.3,
                    delta=0.01, eta=0.032,
                    mu2=1.86/1000, sigma2=0.002177598558933893, rho=.9, max_iter=10_000, tol=1e-8):
    error = 1
    episode = 0
    while error > tol:
        phi_grid_old = phi_grid.copy()
        phi_grid = pde_solver(phi_grid_old,  z_grid, y_grid, terminal_condition, dlambdaMat, epsilon)
        error = np.max(np.abs((phi_grid_old-phi_grid)/epsilon))
        episode += 1
        print('Episode: {}\t error: {}'.format(episode,error))
    return phi_grid

In [104]:
phi = false_transient(V.reshape(len(stateSpace), order="F"), z_grid, y_grid_new, terminal_v, dlambdaMat)

Episode: 1	 error: 0.056292853644813665
Episode: 2	 error: 0.04409892224214263
Episode: 3	 error: 0.06261116099389791
Episode: 4	 error: 0.08898455651251157
Episode: 5	 error: 0.12650907330002192
Episode: 6	 error: 0.1798765009210522
Episode: 7	 error: 0.25577688586148417
Episode: 8	 error: 0.36370957112700203
Episode: 9	 error: 0.5171728181864571
Episode: 10	 error: 0.735364066136132
Episode: 11	 error: 1.0455718443160298
Episode: 12	 error: 1.4866866261721416
Episode: 13	 error: 2.1140424513384835
Episode: 14	 error: 3.0063424555423808
Episode: 15	 error: 4.275534051609011
Episode: 16	 error: 6.08085544382598
Episode: 17	 error: 8.648815966423653
Episode: 18	 error: 12.301617282561537
Episode: 19	 error: 17.49758159719752
Episode: 20	 error: 24.888659617999405
Episode: 21	 error: 35.40223408130623
Episode: 22	 error: 50.35748314854871
Episode: 23	 error: 71.63090438187368
Episode: 24	 error: 101.89176843661429
Episode: 25	 error: 144.9370460469586
Episode: 26	 error: 206.167833303315

KeyboardInterrupt: 